# The electrostatic potential of free atoms

In [ ]:
import sys
sys.path.insert(0, '/home/misa/git_repositories/APDFT/prototyping/atomic_energies/')

import numpy as np
import scipy as sc
from parse_density_files import CUBE
import alchemy_tools2 as at
import qml_interface as qmi
import utils_qm as uq
import glob
from matplotlib import pyplot as plt

from ase.units import Bohr

In [ ]:
def get_paths():
    pass

def load_cube_data(paths_cubes):
    """
    returns the data necessary to calculate the atomic energies from the cube-files
    for different lambda values
    
    paths_cubes: paths to cubes files
    densities: densities given in different cube files
    lam_vals: lambda value for cube file, obtained by parsing filename
    nuclei: charges and coordinates of the nuclei
    gpts: the gridpoints where density values are given
    """
    
    densities = []
    nuclei = None # nuclear charges and their positions
    gpts = None # gridpoints where density values are given
    h_matrix = np.zeros((3,3)) # needed for the calculation of the distance of the nuclei to the gridpoints with MIC
    
    for idx, path in enumerate(paths_cubes):
        cube = CUBE(path)
        
        densities.append(cube.data_scaled) # density
        
        if idx==len(paths_cubes)-1:
            nuclei = cube.atoms
            gpts = cube.get_grid()
            h_matrix = [cube.X*cube.NX, cube.Y*cube.NY, cube.Z*cube.NZ]
    
    return(densities, nuclei, gpts, h_matrix)

def generate_ueg(num_elec, shape):
    volume = 1
    for s in shape:
        volume *= s
    return(np.full(shape, num_elec/volume))

def load_free_atoms_densities(element):
    if element == 'H':
        #################################
        #          HYDROGEN
        #################################
        # load paths to H
        paths = ['/home/misa/APDFT/prototyping/atomic_energies/results/slice_ve38/free_atoms/H/ve_0p6/DENSITY.cube',
                 '/home/misa/APDFT/prototyping/atomic_energies/results/slice_ve38/free_atoms/H/ve_0p7/DENSITY.cube',
                 '/home/misa/APDFT/prototyping/atomic_energies/results/slice_ve38/free_atoms/H/ve_0p8/DENSITY.cube',
                 '/home/misa/APDFT/prototyping/atomic_energies/results/slice_ve38/free_atoms/H/ve_1/DENSITY.cube']
        # generate lam_vals H
        lam_vals = np.array([0.0, 23/38, 0.7, 30/38, 1])
        # load cube files
        densities, nuclei, gpts, h_matrix = load_cube_data(paths)
        shape = densities[0].shape
        # generate cube file for ueg H
        ueg = generate_ueg(1, shape)

    elif element == 'C':
        #################################
        #          CARBON
        #################################
        # load paths to C
        paths = ['/home/misa/APDFT/prototyping/atomic_energies/results/slice_ve38/free_atoms/C/ve_0p2/DENSITY.cube',
                 '/home/misa/APDFT/prototyping/atomic_energies/results/slice_ve38/free_atoms/C/ve_0p8/DENSITY.cube',
                 '/home/misa/APDFT/prototyping/atomic_energies/results/slice_ve38/free_atoms/C/ve_1/DENSITY.cube']
        # generate lam_vals C
        lam_vals = np.array([0.0, 8/38, 30/38, 1])
        # load cube files
        densities, nuclei, gpts, h_matrix = load_cube_data(paths)
        shape = densities[0].shape
        # generate cube file for ueg C
        ueg = generate_ueg(4, shape)

    elif element == 'N':
        #################################
        #          NITROGEN
        #################################
        # load paths to N
        paths = ['/home/misa/APDFT/prototyping/atomic_energies/results/slice_ve38/free_atoms/N/ve_0p4/DENSITY.cube',
                 '/home/misa/APDFT/prototyping/atomic_energies/results/slice_ve38/free_atoms/N/ve_1/DENSITY.cube']
        # generate lam_vals N
        lam_vals = np.array([0.0, 15/38, 1])
        # load cube files
        densities, nuclei, gpts, h_matrix = load_cube_data(paths)
        shape = densities[0].shape
        # generate cube file for ueg N
        ueg = generate_ueg(5, shape)

    elif element == 'O':
        # load paths to O
        paths = ['/home/misa/APDFT/prototyping/atomic_energies/results/slice_ve38/free_atoms/O/ve_0p2/DENSITY.cube',
                '/home/misa/APDFT/prototyping/atomic_energies/results/slice_ve38/free_atoms/O/ve_0p4/DENSITY.cube',
                '/home/misa/APDFT/prototyping/atomic_energies/results/slice_ve38/free_atoms/O/ve_0p8/DENSITY.cube',
                '/home/misa/APDFT/prototyping/atomic_energies/results/slice_ve38/free_atoms/O/ve_1/DENSITY.cube']
        # generate lam_vals O
        lam_vals = np.array([0.0, 8/38, 15/38, 30/38, 1])
        # load cube files
        densities, nuclei, gpts, h_matrix = load_cube_data(paths)
        shape = densities[0].shape
        # generate cube file for ueg O
        ueg = generate_ueg(6, shape)

    densities.insert(0, ueg)
    return(densities, lam_vals)

def get_elec_pot(dens, gpts, dist, h_mat):
    """
    # calculate electrostatic potential
    """
    dvecs = dist/np.sqrt(3)
    center = 10.0/Bohr
    pos_R_I = []

    for dvec in dvecs:
        pos_R_I.append([center+dvec, center+dvec, center+dvec])
    pos_R_I = np.array(pos_R_I)

    # calculate integrals
    integrals = []
    for pos in pos_R_I:
        integrals.append(get_alchpot(dens, pos, gpts, h_mat))
    return(integrals)

def get_alchpot(density, nuc_pos, meshgrid, h_matrix):
    dist = at.distance_MIC2(nuc_pos, meshgrid, h_matrix)
    alch_pot = -(density/dist).sum()
    return(alch_pot)

### Maximum distance between two atoms in a molecule

In [ ]:
def get_largest_distance(coords):
    distances = sc.spatial.distance.cdist(coords, coords)
    max_dist = []
    for d in distances:
        max_dist.append(np.max(d))
    return(np.amax(max_dist))

In [ ]:
paths = qmi.wrapper_alch_data()
data, msize = qmi.load_alchemy_data(paths)

largest_dist = []
for mol in data:
    largest_dist.append(get_largest_distance(mol[:,1:4]))

In [ ]:
np.amax(largest_dist)

In [ ]:
np.linalg.norm( np.array([20,20,20])-np.array([10,10,10]) )

### Calculate averaged density

In [ ]:
densities, nuclei, gpts, h_matrix = load_cube_data(['/home/misa/APDFT/prototyping/atomic_energies/results/slice_ve38/free_atoms/H/ve_0p6/DENSITY.cube'])

densities, lam_vals = load_free_atoms_densities('C')
average_density = at.integrate_lambda_density(densities, lam_vals)

In [ ]:
for i in range(len(densities)):
    plt.plot(np.linspace(0, 20, densities[i].shape[0]), densities[i].sum((2,0)))

In [ ]:
# # calculate averaged density
# elements = ['H', 'C', 'N', 'O']

# for el in elements:
#     densities, lam_vals = load_free_atoms_densities(el)
#     average_density = at.integrate_lambda_density(densities, lam_vals)
#     uq.save_obj(average_density, f'/home/misa/APDFT/prototyping/atomic_energies/results/slice_ve38/free_atoms/av_dens_{el}')

In [ ]:
densities, nuclei, gpts, h_matrix = load_cube_data(['/home/misa/APDFT/prototyping/atomic_energies/results/slice_ve38/free_atoms/H/ve_0p6/DENSITY.cube'])

dist = np.arange(0, 17.5/Bohr, 0.25)
poly_objs = []
ints = []
for el in ['H', 'C', 'N', 'O']:
    av_dens = uq.load_obj(f'/home/misa/APDFT/prototyping/atomic_energies/results/slice_ve38/free_atoms/av_dens_{el}')
    integrals = get_elec_pot(av_dens, gpts, dist, h_matrix)
    ints.append(integrals)
    # fit potential
    poly_obj = sc.interpolate.CubicSpline(dist, integrals)
    poly_objs.append(poly_obj)
    # save result
    save_path = f'/home/misa/APDFT/prototyping/atomic_energies/results/slice_ve38/free_atoms/{el}/'
    uq.save_obj(poly_obj, save_path+f'poly_{el}_recalculated')

In [ ]:
dens = []
for el in ['H', 'C', 'N', 'O']:
    dens.append(uq.load_obj(f'/home/misa/APDFT/prototyping/atomic_energies/results/slice_ve38/free_atoms/av_dens_{el}'))

In [ ]:
dens = [average_density]
nuc_pos = np.array([10,10,10])/Bohr
for d in dens:
    print(get_alchpot(d, nuc_pos, gpts, h_matrix))

In [ ]:
average_density.sum()

In [ ]:
# plot fits
fig, ax = plt.subplots(1,1)
plt.rcParams['font.size'] = 20

for el, poly_obj, integrals in zip(['H', 'C', 'N', 'O'], poly_objs, ints):
    eval_x = np.arange(0, 21/Bohr, 0.1)
    integral_fit = poly_obj.__call__(eval_x)
    ax.plot(dist, integrals, '-o', label = el)
    ax.plot(eval_x, integral_fit)
    
ax.set_xlabel(r'$|\vec{R}_J - \vec{R}_I|$ (Bohr)')
ax.set_ylabel(r'$\int d\vec{r} \frac{\tilde{\rho}_{J, \rm{at}}}{|\vec{r}-\vec{R}_I|}$')
ax.legend()

In [ ]:
# compare fine grid and recalculated to see if density were switched
base_path = f'/home/misa/APDFT/prototyping/atomic_energies/results/slice_ve38/free_atoms/'
poly_recalculated = []
poly_fine = []

elements =  ['H', 'C', 'N', 'O']
for el in elements:
    poly_recalculated.append(uq.load_obj(base_path+f'{el}/poly_{el}_recalculated'))
    poly_fine.append(uq.load_obj(base_path+f'{el}/poly_{el}_fine'))
    
fig, ax = plt.subplots(2,2)
plt.rcParams['figure.figsize'] = [10.0, 10.0]
import itertools as it
idx = it.product((0,1), (0,1))
for i, pr, pf in zip(idx, poly_recalculated, poly_fine):
    eval_x = np.arange(0, 21/Bohr, 0.1)
    y_recalculated = pr.__call__(eval_x)
    y_fine = pf.__call__(eval_x)
    ax[i].plot(eval_x, y_recalculated, label = 're')
    ax[i].plot(eval_x, y_fine, '--', label = 'old')
    ax[i].legend()

# Calculate the atomic alchemical binding potential from the fit

In [ ]:
import numpy as np
from ase.units import Bohr

import sys
sys.path.insert(0, '/home/misa/git_repositories/APDFT/prototyping/atomic_energies/')

import numpy as np
import qml_interface as qmi
import utils_qm as uq
import geometry_euston as ge
import glob

In [ ]:
def get_alchpot_free_fit(molecule, poly_obs):
    """
    calculates the alchemical binding potential for every atom in molecule
    """
    # construct h matrix
    h_mat = ge.abc_to_hmatrix(20,20,20,90,90,90)/Bohr
    # get alchpot of free atoms
    alch_pot_free = []
    for atom_I in molecule:
        alch_pot_free_I = 0
        for atom_J in molecule:
            dist = ge.distance_pbc(atom_I[1:4], atom_J[1:4], h_mat)
            nuc_charge = atom_J[0]
            poly_obj = poly_obs[nuc_charge] # dictionary
            elec_pot = poly_obj.__call__([dist])[0]
            alch_pot_free_I += elec_pot
        alch_pot_free.append(alch_pot_free_I)
        
    alch_pot_free = np.array(alch_pot_free)
#     alch_pot_bind = molecule[:, 4] - alch_pot_free
    return(alch_pot_free)

def get_alch_bind_pot(molecule, poly_obs):
    """
    calculates the alchemical binding potential for every atom in molecule
    """
    alch_pot_free = get_alchpot_free_fit(molecule, poly_obs)
    alch_pot_bind = molecule[:, 4] - alch_pot_free
    return(alch_pot_bind)

def get_element_charge(el):
    if el == 'H':
        return(1)
    elif el == 'C':
        return(6)
    elif el == 'N':
        return(7)
    elif el == 'O':
        return(8)
    else:
        raise ValueError('Symbol for given charge not available')

def load_poly_objs(paths_poly):
    poly_objs = dict()
    for p in paths_poly:
        el = p.split('/')[-1].split('_')[-1]
        charge = get_element_charge(el)
        poly_objs[charge] = uq.load_obj(p)
    return(poly_objs)

In [ ]:
# load molecule data and electrostatic potentials
paths = qmi.wrapper_alch_data()
basepath = [p.strip('atomic_energies_with_mic.txt') for p in paths]
data, msize = qmi.load_alchemy_data(paths)

# paths_poly = glob.glob('/home/misa/APDFT/prototyping/atomic_energies/results/slice_ve38/free_atoms/*/poly_*_fine')
# poly_objs = load_poly_objs(paths_poly)
paths_poly = ['/home/misa/APDFT/prototyping/atomic_energies/results/slice_ve38/free_atoms/H/poly_H_fine',
              '/home/misa/APDFT/prototyping/atomic_energies/results/slice_ve38/free_atoms/C/poly_C_fine',
              '/home/misa/APDFT/prototyping/atomic_energies/results/slice_ve38/free_atoms/N/poly_N_fine',
              '/home/misa/APDFT/prototyping/atomic_energies/results/slice_ve38/free_atoms/O/poly_O_fine']

poly_objs = dict()
for k,p in zip([1.0, 6.0, 7.0, 8.0], paths_poly):
    poly_objs[k] = uq.load_obj(p)

In [ ]:
# calculate potentials
for molecule, path in zip(data, basepath):
    # calculate binding values
    alch_pot_bind = get_alch_bind_pot(molecule, poly_objs)
    atomic_atomisation = alch_pot_bind*molecule[:,0]
    # save data
    save_arr = np.concatenate( (molecule[:,0:5], np.array([alch_pot_bind]).T, np.array([atomic_atomisation]).T), axis=1)
    h = 'charge\t x_coord\t y_coord\t z_coord\t alchemical_potential\t alchemical_binding_potential\t atomisation_energies'
    np.savetxt(path+'atomic_binding_energies2.txt', save_arr, delimiter='\t', header=h)

# Calculate the alchemical potential of the free atoms explicitly

In [1]:
import sys
sys.path.insert(0, '/home/misa/git_repositories/APDFT/prototyping/atomic_energies/')
sys.path.insert(0, '/home/misa/git_repositories/APDFT/prototyping/atomic_energies/hitp')

from ase.units import Bohr
import numpy as np
import qml_interface as qmi
import alchemy_tools2 as at
import utils_qm as uq
from find_converged import concatenate_files
import os

def get_meshgrid(lx, ly, lz, nx, ny, nz):
    """
    returns the coordinates of the grid points where the density values are given as a meshgrid
    works so far only for orthogonal coordinate axes
    """
    # length along the axes
    l_x = lx[0]*nx
    l_y = ly[1]*ny
    l_z = lz[2]*nz
    # gpts along every axis
    x_coords = np.linspace(0, l_x-lx[0], nx)
    y_coords = np.linspace(0, l_y-ly[1], ny)
    z_coords = np.linspace(0, l_z-lz[2], nz)
    # create gridpoints
    meshgrid = np.meshgrid(x_coords, y_coords, z_coords, indexing='ij')
    return(meshgrid)

def get_alchpot_free(nuclei, densities_free_atoms, meshgrid, h_matrix, pos_free_atom = np.array([10.0, 10, 10])/Bohr):
    """
    calculate alchemical potential of free atoms at position of every nucleus
    length units should be given in Bohr
    """
    alch_pot_free = []
    for atom_I in nuclei:
        alch_pot_free_I = 0
        for atom_J in nuclei:
            # get density of free atom J
            nuc_charge = atom_J[0]
            density_fa = densities_free_atoms[nuc_charge]
            # calculate distance of R_I to all gridpoints (shift because free J is in center of box)
            s = (pos_free_atom - atom_J[1:4])
            RI_prime = atom_I[1:4] + s
            dist = at.distance_MIC2(RI_prime, meshgrid, h_matrix)
            # integrate
            elec_pot = -(density_fa/dist).sum()
            alch_pot_free_I += elec_pot
        alch_pot_free.append(alch_pot_free_I)  
    return(np.array(alch_pot_free))

In [2]:
# load average density of free atoms
base_path = '/home/misa/APDFT/prototyping/atomic_energies/results/slice_ve38/free_atoms/'
densities_free_atoms = {1.0:None, 6.0:None, 7.0:None, 8.0:None}
elements = ['H', 'C', 'N', 'O']
for el, k in zip(elements, densities_free_atoms):
    densities_free_atoms[k] = uq.load_obj(base_path + f'av_dens_{el}')

# get meshgrid and h_matrix
#  cell_parameters
nx, ny, nz = densities_free_atoms[1.0].shape
lx, ly, lz = np.array([[20/(Bohr*nx), 0 , 0], [0, 20/(Bohr*ny), 0], [0, 0, 20/(Bohr*nz)]])

h_matrix = [lx*nx, ly*ny, lz*nz]
meshgrid = get_meshgrid(lx, ly, lz, nx, ny, nz)


# paths to the compounds
dirs = concatenate_files(['/home/misa/APDFT/prototyping/atomic_energies/results/slice_ve38/paths_atomic_energies'])

for compound_path in dirs:
    # load ml data files instead of cuves
    molecule = np.loadtxt(compound_path + 'atomic_energies_with_mic.txt')
    alchpot_free = get_alchpot_free(molecule, densities_free_atoms, meshgrid, h_matrix)

    alchpot_bind = molecule[:, 4] - alchpot_free
    atomic_atomisation_pbc = alchpot_bind*molecule[:,0]
    
    # write atomic energies and alchemical potentials to file
    store = np.array([molecule[:,0], molecule[:,1], molecule[:,2], molecule[:,3], molecule[:, 4], alchpot_free, alchpot_bind, atomic_atomisation_pbc]).T
    header = 'charge\t x_coord\t y_coord\t z_coord\t alchemical_potential\t alch_pot_free\t alch_pot_bind\t atomic_atomisation_pbc'
    save_dir = os.path.join(compound_path, 'atomic_binding_energies_explicit.txt')
    np.savetxt(save_dir, store, delimiter='\t', header = header)# 

In [5]:
explicit = []
fit = []
for d in dirs[3:6]:
    explicit.append(np.loadtxt(d + 'atomic_binding_energies_explicit.txt'))
    fit.append(np.loadtxt(d + 'atomic_binding_energies2.txt'))

In [6]:
for e, f in zip(explicit, fit):
    print(e[:,6] - f[:,5])

[-0.01410313 -0.01669352 -0.01373    -0.02318162 -0.04107668 -0.04622965
 -0.04136944  0.0411945  -0.05323291 -0.06125237  0.04119506 -0.05851647
  0.05515931  0.05280735  0.01387246 -0.01520669 -0.06439466 -0.04929259
 -0.01230325 -0.00291437]
[-0.00593783 -0.00651419 -0.00328647  0.00224592 -0.02010216 -0.02845616
 -0.03524938  0.0459867  -0.04462704 -0.04675437  0.04729163 -0.05077733
  0.0581987   0.05179985  0.03122889 -0.00181674 -0.04965625 -0.06464185]
[-0.00066476 -0.00649659 -0.007663   -0.01460567  0.0100618   0.05812744
 -0.02018221  0.05337136 -0.0383827   0.03793875 -0.0088108  -0.02698075
 -0.03041611  0.06116724 -0.02232515  0.035696  ]
